In [ ]:
import numpy as np
import nibabel as nb
from scipy import ndimage

### Load labels and data

In [ ]:
labels = list(np.genfromtxt("/home/julia/data/gradients/atlas/cortex/cortex_idx.csv"))
atlas = nb.load("/home/julia/data/gradients/data_jo/ABI_atlas_reduced_25um.nii.gz")
data = atlas.get_data()

### Make first cortex mask from label indices 

We could have also just downloaded and registered this from Allen API

In [ ]:
mask = np.zeros(data.shape)
for l in labels: 
    mask[np.where(data==int(l))]=1
nb.save(nb.Nifti1Image(mask, img.affine, img.header), 
        "/home/julia/data/gradients/atlas/cortex/cortex_mask_25um.nii.gz")

### Find the missing piece of cortex

A mix of manual thresholding and mask drawing and image clustering with scipy

In [ ]:
x = nb.load("/home/julia/data/gradients/atlas/cortex/x_25um.nii.gz").get_data()
y = ndimage.label(x)
nb.save(nb.Nifti1Image(y[0], img.affine, img.header), 
        "/home/julia/projects/gradients/atlas/cortex/y_25um.nii.gz")

In [ ]:
z = np.zeros(x.shape)
z[np.where(y[0]==1)]=1
z[np.where(y[0]==12)]=1
nb.save(nb.Nifti1Image(z, img.affine, img.header), 
        "/home/julia/data/gradients/atlas/cortex/z_25um.nii.gz")

In [ ]:
z_new=nb.load("/home/julia/data/gradients/atlas/cortex/z_25um_copy.nii.gz").get_data()
q = ndimage.label(z_new)
nb.save(nb.Nifti1Image(q[0], img.affine, img.header), 
        "/home/julia/data/gradients/atlas/cortex/q_25um.nii.gz")

In [ ]:
j = np.zeros(z.shape)
j[np.where(q[0]==2)] = 1
j[np.where(q[0]==10)] = 1
nb.save(nb.Nifti1Image(j, img.affine, img.header), 
        "/home/julia/data/gradients/atlas/cortex/j_25um.nii.gz")

### Fix cortex with the missing piece, smooth and dilate

In [ ]:
cortex=nb.load("/home/julia/data/gradients/atlas/cortex/cortex_mask_25um.nii.gz").get_data()

In [ ]:
cortex_fixed=np.zeros(j.shape)
cortex_fixed[np.where(cortex>0)]=1
cortex_fixed[np.where(j>0)]=1

In [ ]:
nb.save(nb.Nifti1Image(cortex_fixed, img.affine, img.header), 
        "/home/julia/data/gradients/atlas/cortex/cortex_mask_25um.nii.gz")

In [ ]:
!fslmaths cortex_mask_25um.nii.gz -s 0.2 -thr 0.4 -bin cortex_mask_25um.nii.gz
!fslmaths cortex_mask_25um.nii.gz -dilM -dilM cortex_mask_25um_dilM2.nii.gz

### Make a mask that contains all structures

Again could have been downloaded from Allen API

In [ ]:
ven = nb.load("/home/julia/data/gradients/atlas/mask_VEN_bin_25um.nii.gz")
wm = nb.load("/home/julia/data/gradients/atlas/mask_WM_bin_25um.nii.gz")

In [ ]:
allmask = np.zeros(ven.shape)
allmask[np.where(ven.get_data()!=0)] = 1
allmask[np.where(wm.get_data()!=0)] = 1
allmask[np.where(atlas.get_data()!=0)] = 1

Fill in some missing labels

In [ ]:
atlas_full = nb.load("/home/julia/data/gradients/atlas/ABI_atlas_25um.nii.gz")
new_labels = [61, 72, 73, 290, 301, 302]
for l in new_labels: 
    allmask[np.where(atlas.get_data()==l)]=1

nb.save(nb.Nifti1Image(allmask, ven.affine, ven.header), 
        "/home/julia/data/gradients/atlas/cortex/all_mask_25um.nii.gz")

### Make a mask containing everything but cortex

In [ ]:
cortex = nb.load("/home/julia/data/gradients/atlas/cortex/cortex_mask_25um.nii.gz")
inside = allmask_new - cortex.get_data()
inside[np.where(inside==-1)]=0
nb.save(nb.Nifti1Image(inside, cortex.affine, cortex.header), 
        "/home/julia/data/gradients/atlas/make_cortex/cortex/inside_mask_25um.nii.gz")

### Make fake white matter and white/grey matter masks 

In [ ]:
cortex_dil = nb.load("/home/julia/data/gradients/atlas/cortex/cortex_mask_25um_dilM2.nii.gz")
wm_fake = np.zeros(cortex.get_data().shape)
wm_fake[np.where(inside>0)] = 1
wm_fake[np.where(cortex.get_data()>0)] = 0
wm_fake *= cortex_dil.get_data()

In [ ]:
nb.save(nb.Nifti1Image(wm_fake, cortex.affine, cortex.header), 
        "/home/julia/projects/gradients/atlas/cortex/wm_fake_25um.nii.gz")

In [ ]:
!fslmaths wm_fake_25um.nii.gz -dilM -dilM -dilM -dilM wm_fake_25um_dilM4.nii.gz
!fslmaths wm_fake_25um_dilM4.nii.gz -dilM -dilM -dilM -dilM wm_fake_25um_dilM8.nii.gz

In [ ]:
!fslmaths wm_fake_25um_dilM8.nii.gz -add cortex_mask_25um.nii.gz -bin wmgm_mask.nii.gz
!fslmaths wm_fake_25um_dilM8.nii.gz -sub cortex_mask_25um.nii.gz -thr 0 wm_mask.nii.gz